In [2]:
# Retina quality plots
%config InlineBackend.figure_format = 'retina'

In [3]:
DATA_DIR = '../../data/reduced/small/'

In [4]:
import pandas as pd

train_data = pd.read_csv(DATA_DIR + 'PCA_train.csv')

train_data.info()
train_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35187 entries, 0 to 35186
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   0         35187 non-null  float64
 1   1         35187 non-null  float64
 2   phishing  35187 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 824.8 KB


,0,1,phishing
0,228987.058114,-7577.447893,1
1,-18079.501237,-4591.866489,1
2,-7855.407037,-4726.854052,1
3,229855.757883,5730.655682,1
4,-19920.590656,-4432.011638,0


In [5]:
valid_data = pd.read_csv(DATA_DIR + 'PCA_valid.csv')

valid_data.info()
valid_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11729 entries, 0 to 11728
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   0         11729 non-null  float64
 1   1         11729 non-null  float64
 2   phishing  11729 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 275.0 KB


,0,1,phishing
0,-19913.487308,-4573.678818,1
1,-18080.461643,-4663.846316,0
2,-19152.980778,-1755.291659,0
3,13510.492490,6797.767366,1
4,-4298.720730,-1475.949864,0


In [6]:
test_data = pd.read_csv(DATA_DIR + 'PCA_test.csv')

test_data.info()
test_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11729 entries, 0 to 11728
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   0         11729 non-null  float64
 1   1         11729 non-null  float64
 2   phishing  11729 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 275.0 KB


,0,1,phishing
0,18815.156711,-1661.175732,1
1,-33252.125551,-4689.002127,1
2,-14168.852892,-2852.311369,0
3,-16619.548133,16682.891987,1
4,21832.273241,1944.771778,0


In [7]:
TARGET_COL_NAME = 'phishing'

In [10]:
X_train = train_data.drop(TARGET_COL_NAME, axis=1)
y_train = train_data[TARGET_COL_NAME]

In [11]:
y_train.value_counts()

1    18388
0    16799
Name: phishing, dtype: int64

In [12]:
X_valid = valid_data.drop(TARGET_COL_NAME, axis=1)
y_valid = valid_data[TARGET_COL_NAME]

In [13]:
y_valid.value_counts()

1    6129
0    5600
Name: phishing, dtype: int64

In [14]:
X_test = test_data.drop(TARGET_COL_NAME, axis=1)
y_test = test_data[TARGET_COL_NAME]

In [15]:
y_test.value_counts()

1    6130
0    5599
Name: phishing, dtype: int64

## Hyperparameter tuning

In [1]:
from sklearn import svm
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score

# Grid search over hyperparameters
kernel = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
print('1')
clf = svm.SVC()
print('1')
clf.fit(X_train, y_train)
print('1')
y_pred = clf.predict(X_valid)

print('Validation data - Accuracy: {:.3f}, Balanced accuracy: {:.3f}, F1: {:.3f}, Precision: {:.3f}, Recall: {:.3f}'
      .format(accuracy_score(y_valid, y_pred),
              balanced_accuracy_score(y_valid, y_pred),
              f1_score(y_valid, y_pred),
              precision_score(y_valid, y_pred),
              recall_score(y_valid, y_pred)))
print('-' * 20)

# for k in kernel:
#     print('Kernel is {}'.format(k))

#     clf = svm.SVC(kernel=k)
#     clf.fit(X_train, y_train)

#     y_pred = clf.predict(X_valid)

#     print('Validation data - Accuracy: {:.3f}, Balanced accuracy: {:.3f}, F1: {:.3f}, Precision: {:.3f}, Recall: {:.3f}'
#           .format(accuracy_score(y_valid, y_pred),
#                   balanced_accuracy_score(y_valid, y_pred),
#                   f1_score(y_valid, y_pred),
#                   precision_score(y_valid, y_pred),
#                   recall_score(y_valid, y_pred)))
#     print('-' * 20)

1
1


NameError: name 'X_train' is not defined